### Archived Statements - part 3

**Target:**

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516+ 2,159,986,344 = 5,399,965,860

 total: 7,019,952,410

<img src="monitoring/7B-index.png"/>

**Process:**

 - bulk import via stream consumer of import/statements_archive3
 - use externalScroller to bulk index at the same time

During the last // import we saw that the throughput ended up dropping to 20Kdocs/s, so the gain of having 2 injectors running concurrently does not seem that useful.
Let's use the remaining MongoDB throughput to do the indexing and avoid having to redo a full scroll+reindex.

#### Run the imports

In [ ]:
!import.sh -o import -t 24 -l import/statements_archive3 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step3.log

Started BulkIndex using ExternalScroller: ee492d2f-112a-4f84-90ef-e63ae146c886

In [5]:
# check status
! cid="ee492d2f-112a-4f84-90ef-e63ae146c886"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

{"entity-type":"bulkStatus","commandId":"ee492d2f-112a-4f84-90ef-e63ae146c886","state":"SCROLLING_RUNNING","processed":434609856,"error":false,"errorCount":0,"total":0,"action":"index","username":"nco-admin","submitted":"2020-09-20T13:13:27.362Z","scrollStart":"2020-09-20T13:13:27.384Z","scrollEnd":null,"processingStart":null,"processingEnd":null,"completed":null,"processingMillis":0}

The first run failed because of OOM: see https://jira.nuxeo.com/browse/SUPINT-1789

In [6]:
# cancel previously running BAF
! cid="ee492d2f-112a-4f84-90ef-e63ae146c886"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -X PUT -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid/abort

{"entity-type":"bulkStatus","commandId":"ee492d2f-112a-4f84-90ef-e63ae146c886","state":"ABORTED","processed":434609856,"error":false,"errorCount":0,"total":0,"action":"index","username":"nco-admin","submitted":"2020-09-20T13:13:27.362Z","scrollStart":"2020-09-20T13:13:27.384Z","scrollEnd":null,"processingStart":null,"processingEnd":null,"completed":null,"processingMillis":0}

In [7]:
!import.sh -o import -t 24 -l import/statements_archive3 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step3-2.log

^C


In [9]:
! bid="d7c79366-b4be-4c7e-9a22-d9ed2fc49bdc"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -X DELETE -u $NXUSER:$NXPWD $INJECTOR/nuxeo/site/api/v1/automation/StreamImporter.runDocumentConsumersEx/@async/$bid

{"entity-type":"exception","status":404,"message":"Execution with id=d7c79366-b4be-4c7e-9a22-d9ed2fc49bdc has completed"}

In [ ]:
!import.sh -o import -t 24 -l import/statements_archive3 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step3-3.log

Result for the importer:

    Consumers status: threads: 24, failure 0, messages committed: 1215511646, elapsed: 89641.75s, throughput: 13559.66 msg/s


In [13]:
!tail logs/import-archives-step3-3.log

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

Started BulkIndex using ExternalScroller: e984584c-6763-4283-874c-506c85385a04


In [11]:
# check status
! cid="e984584c-6763-4283-874c-506c85385a04"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

{"entity-type":"bulkStatus","commandId":"e984584c-6763-4283-874c-506c85385a04","state":"COMPLETED","processed":1215544852,"error":false,"errorCount":0,"total":1215511646,"action":"index","username":"nco-admin","submitted":"2020-09-21T13:29:30.364Z","scrollStart":"2020-09-21T13:29:30.375Z","scrollEnd":"2020-09-22T14:23:32.185Z","processingStart":null,"processingEnd":null,"completed":"2020-09-22T14:23:32.216Z","processingMillis":0}

In [14]:
from dateutil import parser
start = "2020-09-21T13:29:30.364Z"
end = "2020-09-22T14:23:32.216Z"
nbdocs = 1215511646
s = parser.parse(start)
e = parser.parse(end) 
throughput = nbdocs / (e-s).total_seconds()

print('%s docs/s' % format(throughput, ',.2f'))

13,559.64 docs/s


### Gather statistics

In [15]:
!mongo "mongodb+srv://nxbench-2826-archives.vzeqm.mongodb.net/nuxeo" \
  --username $MONGOUSER --password $MONGOPWD \
  ../ToolBox/archives-mongodb-stats.js > snapshots/"mongodb-archives-"`date +%Y%m%d-%H%M%S`.bson

In [16]:
!statfile="snapshots/es-5B-archive-indexing-"`date +%Y%m%d-%H%M%S`.txt; \
 curl "$ES_SERVER" > $statfile; \
 curl "$ES_SERVER/_cat/health?v" >> $statfile ; \
 curl "$ES_SERVER/_cat/nodes?v" >> $statfile; \
 curl "$ES_SERVER/_cat/indices?v" >> $statfile; \
 cat $statfile

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   509  100   509    0     0  11568      0 --:--:-- --:--:-- --:--:-- 11568
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   336  100   336    0     0  14000      0 --:--:-- --:--:-- --:--:-- 14000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1403  100  1403    0     0  28632      0 --:--:-- --:--:-- --:--:-- 29229
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   864  100   864    0     0  20093      0 --:--:-- --:--:-- --:--:-- 20093
{
  "name" : "G4PH9nZ",
  "cluster_name" : "18867088